In [1]:
import sys
import argparse
import numpy as np
from scipy.stats import chi2_contingency
from itertools import product
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)
    
from datetime import datetime
import tempfile
import os
import gzip
import subprocess
import time

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

In [7]:
t= (1,4)
np.prod(t)

4

In [2]:
from tools.mcounter_tools import (
    read_vcf_allel, ind_assignment_scatter_v1, MC_sample_matrix_v1,
    heatmap_v2, ind_assignment_SFS, read_windows_SFS
)

In [3]:
#from tools.SLiM_pipe_tools import mutation_counter_launch
import re
import pandas as pd


## directories
main_dir= os.getcwd() + '/'
sims_dir= main_dir + 'mutation_counter/data/sims_dem/'
diffs= False
frequency_range= [0,1]
args_present= False

data_kmer, data= read_windows_SFS(diffs= diffs, frequency_range= frequency_range,indfile= 'ind_assignments.txt', outemp= 'ind_assignments{}.txt',
                    sim_dir= sims_dir, muted_dir= 'mutation_counter/data/mutation_count/',
                    outlog= 'indy.log', row= 24,col= 4, single= True, exclude= False, args= args_present)



missing: 0, no vcf: 89
31
time elapsed: 390.0622878074646s


In [12]:

####
def get_mutations(bases= 'ACGT',ksize= 3):
    '''return list of possible kmer mutations'''
    
    mutations=[]
    
    base_set= [bases]*ksize

    for trimer in product(*base_set):
        for base in bases:
            if trimer[int(ksize / 2)] != base:
                mutations.append((''.join(trimer), base))
    
    return mutations


def kmer_comp_index(mutations):
    ''' return nested dictionaries of kmer mutations w/ index'''
    kmers= {}
    kmer_idx= {}
    d= 0
    for kmer in mutations:

        comp= get_complement(kmer[0]) + get_complement(kmer[1])
        comp= ''.join(comp)
        kmer= ''.join(kmer)
        
        if comp in kmers.keys():
            idx= kmers[comp]
            kmers[kmer]= idx
            kmer_idx[idx].append(kmer)
        else:
            kmers[kmer]= len(kmer_idx)
            kmer_idx[len(kmer_idx)]= [kmer]

        d += 1
    
    return kmers, kmer_idx


def kmer_mut_index(mutations):
    '''produce nested dictionary of nucs for a particular mutation list'''
    mut_lib= recursively_default_dict()
    
    for mut in range(len(mutations)):
        trimer= ''.join(mutations[mut])
        get_by_path(mut_lib, trimer[:-1])[trimer[-1]]= mut
    
    return mut_lib




def get_complement(kmer):
    '''Return complement of a given kmer'''
    complements= {
        'A': 'T',
        'T': 'A',
        'C': 'G',
        'G': 'C'
    }
    
    comp= [complements[x] for x in kmer][::-1]
    return comp

def get_by_path(root, items):
    """Access a nested object in root by item sequence."""
    return reduce(operator.getitem, items, root)

def set_by_path(root, items, value):
    """Set a value in a nested object in root by item sequence."""
    get_by_path(root, items[:-1])[items[-1]] = value

####


In [15]:
from functools import reduce  # forward compatibility for Python 3
import operator

bases= 'ATCG'
ksize= 3
mutations= get_mutations(bases= bases,ksize= ksize)
kmers, kmer_idx= kmer_comp_index(mutations)

mut_lib= kmer_mut_index(mutations)

In [28]:
t= [kmer_idx[x][0] for x in sorted(kmer_idx.keys())]
grid= np.array(t).reshape(24,4)
grid

array([['AAAT', 'AAAC', 'AAAG', 'AATT'],
       ['AATC', 'AATG', 'AACT', 'AACC'],
       ['AACG', 'AAGT', 'AAGC', 'AAGG'],
       ['ATAA', 'ATAC', 'ATAG', 'ATCA'],
       ['ATCC', 'ATCG', 'ATGA', 'ATGC'],
       ['ATGG', 'ACAA', 'ACAT', 'ACAG'],
       ['ACTA', 'ACTT', 'ACTG', 'ACCA'],
       ['ACCT', 'ACCG', 'ACGA', 'ACGT'],
       ['ACGG', 'AGAA', 'AGAT', 'AGAC'],
       ['AGCA', 'AGCT', 'AGCC', 'AGGA'],
       ['AGGT', 'AGGC', 'TAAT', 'TAAC'],
       ['TAAG', 'TACT', 'TACC', 'TACG'],
       ['TAGT', 'TAGC', 'TAGG', 'TTCA'],
       ['TTCC', 'TTCG', 'TTGA', 'TTGC'],
       ['TTGG', 'TCAA', 'TCAT', 'TCAG'],
       ['TCCA', 'TCCT', 'TCCG', 'TCGA'],
       ['TCGT', 'TCGG', 'TGCA', 'TGCT'],
       ['TGCC', 'TGGA', 'TGGT', 'TGGC'],
       ['CACT', 'CACC', 'CACG', 'CAGT'],
       ['CAGC', 'CAGG', 'CTCA', 'CTCC'],
       ['CTCG', 'CCCA', 'CCCT', 'CCCG'],
       ['CCGA', 'CCGT', 'CCGG', 'CGCA'],
       ['CGCT', 'CGCC', 'GACT', 'GACC'],
       ['GACG', 'GCCA', 'GCCT', 'GCCG']], dtype='<U4')

In [34]:
grid_to_list= grid.reshape(1,np.prod(grid.shape))[0]
grid_to_list

array(['AAAT', 'AAAC', 'AAAG', 'AATT', 'AATC', 'AATG', 'AACT', 'AACC',
       'AACG', 'AAGT', 'AAGC', 'AAGG', 'ATAA', 'ATAC', 'ATAG', 'ATCA',
       'ATCC', 'ATCG', 'ATGA', 'ATGC', 'ATGG', 'ACAA', 'ACAT', 'ACAG',
       'ACTA', 'ACTT', 'ACTG', 'ACCA', 'ACCT', 'ACCG', 'ACGA', 'ACGT',
       'ACGG', 'AGAA', 'AGAT', 'AGAC', 'AGCA', 'AGCT', 'AGCC', 'AGGA',
       'AGGT', 'AGGC', 'TAAT', 'TAAC', 'TAAG', 'TACT', 'TACC', 'TACG',
       'TAGT', 'TAGC', 'TAGG', 'TTCA', 'TTCC', 'TTCG', 'TTGA', 'TTGC',
       'TTGG', 'TCAA', 'TCAT', 'TCAG', 'TCCA', 'TCCT', 'TCCG', 'TCGA',
       'TCGT', 'TCGG', 'TGCA', 'TGCT', 'TGCC', 'TGGA', 'TGGT', 'TGGC',
       'CACT', 'CACC', 'CACG', 'CAGT', 'CAGC', 'CAGG', 'CTCA', 'CTCC',
       'CTCG', 'CCCA', 'CCCT', 'CCCG', 'CCGA', 'CCGT', 'CCGG', 'CGCA',
       'CGCT', 'CGCC', 'GACT', 'GACC', 'GACG', 'GCCA', 'GCCT', 'GCCG'],
      dtype='<U4')

In [36]:
grid_to_list[21]

'ACAA'

In [23]:
kmer_idx

{0: ['AAAT', 'TTTA'],
 1: ['AAAC', 'TTTG'],
 2: ['AAAG', 'TTTC'],
 3: ['AATT', 'ATTA'],
 4: ['AATC', 'ATTG'],
 5: ['AATG', 'ATTC'],
 6: ['AACT', 'GTTA'],
 7: ['AACC', 'GTTG'],
 8: ['AACG', 'GTTC'],
 9: ['AAGT', 'CTTA'],
 10: ['AAGC', 'CTTG'],
 11: ['AAGG', 'CTTC'],
 12: ['ATAA', 'TATT'],
 13: ['ATAC', 'TATG'],
 14: ['ATAG', 'TATC'],
 15: ['ATCA', 'GATT'],
 16: ['ATCC', 'GATG'],
 17: ['ATCG', 'GATC'],
 18: ['ATGA', 'CATT'],
 19: ['ATGC', 'CATG'],
 20: ['ATGG', 'CATC'],
 21: ['ACAA', 'TGTT'],
 22: ['ACAT', 'TGTA'],
 23: ['ACAG', 'TGTC'],
 24: ['ACTA', 'AGTT'],
 25: ['ACTT', 'AGTA'],
 26: ['ACTG', 'AGTC'],
 27: ['ACCA', 'GGTT'],
 28: ['ACCT', 'GGTA'],
 29: ['ACCG', 'GGTC'],
 30: ['ACGA', 'CGTT'],
 31: ['ACGT', 'CGTA'],
 32: ['ACGG', 'CGTC'],
 33: ['AGAA', 'TCTT'],
 34: ['AGAT', 'TCTA'],
 35: ['AGAC', 'TCTG'],
 36: ['AGCA', 'GCTT'],
 37: ['AGCT', 'GCTA'],
 38: ['AGCC', 'GCTG'],
 39: ['AGGA', 'CCTT'],
 40: ['AGGT', 'CCTA'],
 41: ['AGGC', 'CCTG'],
 42: ['TAAT', 'TTAA'],
 43: ['TAAC', 'TTAG']

71